In [166]:
#apache http library
import requests

#XML parsing
import xml.etree.ElementTree as ET

import os
import pandas as pd
import numpy as np
from matplotlib.patches import Rectangle
from numpy.random import random
import matplotlib.pyplot as plt
import matplotlib.patches as patches
%matplotlib inline
from termcolor import colored

# Obtain line-up data from MLB

The Orioles played the Yankees on August 14th, 2019. Below, we will pull the roster data for that game to find the players on the roster and in the line-up. 

In [167]:
url = 'http://gd2.mlb.com/components/game/mlb/year_2019/month_08/day_14/gid_2019_08_14_balmlb_nyamlb_1/players.xml'
resp = requests.get(url)
print(colored(resp, 'blue'))

<Response [200]>


In [168]:
xmlfile = 'BAL_NYY_Players.xml'
with open(xmlfile, 'wb') as f:
    f.write(resp.content)
statinfo = os.stat(xmlfile)


In [169]:
# pull in players <game><team><player>
tree = ET.parse(xmlfile)
game = tree.getroot()
teams = game.findall('./team')
teams

[<Element 'team' at 0x11b7b1f50>, <Element 'team' at 0x1087c2c50>]

In [170]:
PlayerDict = {}

for team in teams:
    print(team.attrib.get('name'))
    players = team.findall('player')
    for player in players:
        print('', player.attrib.get('id'), player.attrib.get('first'), player.attrib.get('last'))
        PlayerDict[player.attrib.get('id')] = player.attrib.get('first') + ' ' + player.attrib.get('last')

Baltimore Orioles
 542888 Shawn Armstrong
 542947 Richard Bleier
 600474 Pedro Severino
 641820 Trey Mancini
 448801 Chris Davis
 664045 Tom Eshelman
 607054 Jace Peterson
 623993 Anthony Santander
 593679 Gabriel Ynoa
 605156 Aaron Brooks
 547004 Rio Ruiz
 621389 Ty Blach
 592859 Stevie Wilkerson
 600524 Renato Nunez
 621006 Richie Martin
 642082 Chance Sisco
 593643 Hanser Alberto
 643316 Paul Fry
 571710 Mychal Givens
 592879 Asher Wojciechowski
 605164 Dylan Bundy
 612434 Miguel Castro
 607644 John Means
 542340 Jonathan Villar
 605469 Chandler Shepherd
New York Yankees
 641482 Nestor Cortes Jr.
 518934 DJ LeMahieu
 645801 Mike Ford
 593334 Domingo German
 642731 Thairo Estrada
 519222 Austin Romine
 643565 Mike Tauchman
 493603 Adam Ottavino
 664856 Chance Adams
 592450 Aaron Judge
 642528 Jonathan Loaisiga
 572020 James Paxton
 570482 Gio Urshela
 502154 Zack Britton
 547888 Masahiro Tanaka
 457727 Cameron Maybin
 650402 Gleyber Torres
 458731 Brett Gardner
 643338 Chad Green
 59

In [171]:
# Convert to a dataframe
df_players = pd.DataFrame.from_dict(PlayerDict,orient='index')

# Add in a team column
df_players['Team'] = 'BAL'
df_players.at[25:50, 'Team'] = 'NYY'

# Reset index
df_players = df_players.reset_index()
df_players.rename(columns={'index':'playerid'}, inplace=True)
df_players.rename(columns={'playerid':'MLBID'}, inplace=True)

# Convert ID to float
df_players['MLBID'] = df_players['MLBID'].astype(float)

# View data
df_players[23:30]

,MLBID,0,Team
23,542340.0,Jonathan Villar,BAL
24,605469.0,Chandler Shepherd,BAL
25,641482.0,Nestor Cortes Jr.,NYY
26,518934.0,DJ LeMahieu,NYY
27,645801.0,Mike Ford,NYY
28,593334.0,Domingo German,NYY
29,642731.0,Thairo Estrada,NYY


# Obtain Fangraphs Value data

In [172]:
# Import Fangraphs data
df_BAL = pd.read_csv('BAL-PlayerValue.csv')
df_NYY = pd.read_csv('NYY-PlayerValue.csv')

# pd.concat the two dataframes
df_player_value = pd.concat([df_BAL, df_NYY])
df_player_value = df_player_value.reset_index()
df_player_value = df_player_value.drop('index', 1)

In [173]:
df_player_value

,Name,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,playerid
0,Jonathan Villar,6.5,10.5,-6.0,4.5,17.0,-1.4,2.8,22.5,40.8,4.0,$31.7,10071
1,Trey Mancini,28.2,-2.1,-4.1,-9.4,26.1,-13.4,2.7,21.4,36.7,3.6,$28.5,15149
2,Hanser Alberto,-3.1,-1.7,3.1,1.7,-4.7,4.8,2.2,17.3,19.5,1.9,$15.2,11902
3,Renato Nunez,-0.9,0.3,-1.5,-13.2,-0.7,-14.7,2.3,18.8,5.8,0.6,$4.5,14503
4,DJ LeMahieu,30.0,-0.8,3.4,-0.7,29.2,2.8,2.6,20.6,55.1,5.4,$42.8,9874
5,Gleyber Torres,19.1,-0.5,-6.3,3.8,18.7,-2.6,2.4,19.0,37.5,3.6,$29.1,16997
6,Brett Gardner,10.7,4.6,3.0,-0.5,15.2,2.5,2.2,17.3,37.2,3.6,$28.9,9927
7,Luke Voit,16.7,-3.8,-3.9,-9.7,12.9,-13.6,2.0,16.0,17.3,1.7,$13.5,14811


# Obtain Player ID Maps

In [196]:
df_playerid_map = pd.read_csv('SFBB Player ID Map - PLAYERIDMAP.csv')
df_playerid_map = df_playerid_map[['IDFANGRAPHS', 'MLBID']]
df_playerid_map.rename(columns={'IDFANGRAPHS':'playerid'}, inplace=True)

In [197]:
# subset the data to remove player name

df_player_value = df_player_value[['Batting', 'Base Running', 'Fielding', 'Positional', 'Offense', 'Defense', 'League', 'Replacement', 'RAR', 'WAR', 'Dollars', 'playerid']]

In [198]:
df_player_value

,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,playerid
0,6.5,10.5,-6.0,4.5,17.0,-1.4,2.8,22.5,40.8,4.0,$31.7,10071
1,28.2,-2.1,-4.1,-9.4,26.1,-13.4,2.7,21.4,36.7,3.6,$28.5,15149
2,-3.1,-1.7,3.1,1.7,-4.7,4.8,2.2,17.3,19.5,1.9,$15.2,11902
3,-0.9,0.3,-1.5,-13.2,-0.7,-14.7,2.3,18.8,5.8,0.6,$4.5,14503
4,30.0,-0.8,3.4,-0.7,29.2,2.8,2.6,20.6,55.1,5.4,$42.8,9874
5,19.1,-0.5,-6.3,3.8,18.7,-2.6,2.4,19.0,37.5,3.6,$29.1,16997
6,10.7,4.6,3.0,-0.5,15.2,2.5,2.2,17.3,37.2,3.6,$28.9,9927
7,16.7,-3.8,-3.9,-9.7,12.9,-13.6,2.0,16.0,17.3,1.7,$13.5,14811


# Merge Fangraphs Data with PlayerID-Map

In [199]:
df_playerid_map['playerid'] = df_playerid_map['playerid'].astype(str)
df_player_value['playerid'] = df_player_value['playerid'].astype(str)

In [200]:
df_merged = pd.merge(df_playerid_map, df_player_value, on='playerid')

In [201]:
df_merged

,playerid,MLBID,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars
0,11902,NaN,-3.1,-1.7,3.1,1.7,-4.7,4.8,2.2,17.3,19.5,1.9,$15.2
1,9927,458731.0,10.7,4.6,3.0,-0.5,15.2,2.5,2.2,17.3,37.2,3.6,$28.9
2,9874,518934.0,30.0,-0.8,3.4,-0.7,29.2,2.8,2.6,20.6,55.1,5.4,$42.8
3,15149,641820.0,28.2,-2.1,-4.1,-9.4,26.1,-13.4,2.7,21.4,36.7,3.6,$28.5
4,14503,600524.0,-0.9,0.3,-1.5,-13.2,-0.7,-14.7,2.3,18.8,5.8,0.6,$4.5
5,16997,650402.0,19.1,-0.5,-6.3,3.8,18.7,-2.6,2.4,19.0,37.5,3.6,$29.1
6,10071,542340.0,6.5,10.5,-6.0,4.5,17.0,-1.4,2.8,22.5,40.8,4.0,$31.7
7,14811,572228.0,16.7,-3.8,-3.9,-9.7,12.9,-13.6,2.0,16.0,17.3,1.7,$13.5


In [202]:
df_merged.at[0, 'MLBID'] = '593643'

In [203]:
df_merged

,playerid,MLBID,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars
0,11902,593643.0,-3.1,-1.7,3.1,1.7,-4.7,4.8,2.2,17.3,19.5,1.9,$15.2
1,9927,458731.0,10.7,4.6,3.0,-0.5,15.2,2.5,2.2,17.3,37.2,3.6,$28.9
2,9874,518934.0,30.0,-0.8,3.4,-0.7,29.2,2.8,2.6,20.6,55.1,5.4,$42.8
3,15149,641820.0,28.2,-2.1,-4.1,-9.4,26.1,-13.4,2.7,21.4,36.7,3.6,$28.5
4,14503,600524.0,-0.9,0.3,-1.5,-13.2,-0.7,-14.7,2.3,18.8,5.8,0.6,$4.5
5,16997,650402.0,19.1,-0.5,-6.3,3.8,18.7,-2.6,2.4,19.0,37.5,3.6,$29.1
6,10071,542340.0,6.5,10.5,-6.0,4.5,17.0,-1.4,2.8,22.5,40.8,4.0,$31.7
7,14811,572228.0,16.7,-3.8,-3.9,-9.7,12.9,-13.6,2.0,16.0,17.3,1.7,$13.5


# Merge new dataframe with Player data

In [204]:
df_merged = pd.merge(df_merged, df_players, on='MLBID')

In [205]:
df_merged

,playerid,MLBID,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,0,Team
0,11902,593643.0,-3.1,-1.7,3.1,1.7,-4.7,4.8,2.2,17.3,19.5,1.9,$15.2,Hanser Alberto,BAL
1,9927,458731.0,10.7,4.6,3.0,-0.5,15.2,2.5,2.2,17.3,37.2,3.6,$28.9,Brett Gardner,NYY
2,9874,518934.0,30.0,-0.8,3.4,-0.7,29.2,2.8,2.6,20.6,55.1,5.4,$42.8,DJ LeMahieu,NYY
3,15149,641820.0,28.2,-2.1,-4.1,-9.4,26.1,-13.4,2.7,21.4,36.7,3.6,$28.5,Trey Mancini,BAL
4,14503,600524.0,-0.9,0.3,-1.5,-13.2,-0.7,-14.7,2.3,18.8,5.8,0.6,$4.5,Renato Nunez,BAL
5,16997,650402.0,19.1,-0.5,-6.3,3.8,18.7,-2.6,2.4,19.0,37.5,3.6,$29.1,Gleyber Torres,NYY
6,10071,542340.0,6.5,10.5,-6.0,4.5,17.0,-1.4,2.8,22.5,40.8,4.0,$31.7,Jonathan Villar,BAL


In [187]:
# Clean up new data

df_merged = df_merged.drop(0, 1)
df_merged = df_merged.drop([4])

In [192]:
df_merged

,playerid,MLBID,Name,Batting,Base Running,Fielding,Positional,Offense,Defense,League,Replacement,RAR,WAR,Dollars,Team
0,11902,593643.0,Hanser Alberto,-3.1,-1.7,3.1,1.7,-4.7,4.8,2.2,17.3,19.5,1.9,$15.2,BAL
1,9927,458731.0,Brett Gardner,10.7,4.6,3.0,-0.5,15.2,2.5,2.2,17.3,37.2,3.6,$28.9,NYY
2,9874,518934.0,DJ LeMahieu,30.0,-0.8,3.4,-0.7,29.2,2.8,2.6,20.6,55.1,5.4,$42.8,NYY
3,15149,641820.0,Trey Mancini,28.2,-2.1,-4.1,-9.4,26.1,-13.4,2.7,21.4,36.7,3.6,$28.5,BAL
5,16997,650402.0,Gleyber Torres,19.1,-0.5,-6.3,3.8,18.7,-2.6,2.4,19.0,37.5,3.6,$29.1,NYY
6,10071,542340.0,Jonathan Villar,6.5,10.5,-6.0,4.5,17.0,-1.4,2.8,22.5,40.8,4.0,$31.7,BAL
